<a href="https://colab.research.google.com/github/Kaffos/MAGS_Simulation/blob/main/MAGS_Sim_Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Components

### init

In [ ]:
import numpy as np
import pandas as pd

### Antenna

In [ ]:
class Antenna:
    def __init__(self, azimuth_range=(0, 360), elevation_range=(0, 90), slew_rate=1.0):
        self.azimuth_range = azimuth_range
        self.elevation_range = elevation_range
        self.slew_rate = slew_rate
        self.current_azimuth = 0.0
        self.current_elevation = 0.0

    def update_position(self, desired_azimuth, desired_elevation, dt):
        azimuth_error = desired_azimuth - self.current_azimuth
        elevation_error = desired_elevation - self.current_elevation

        azimuth_change = np.clip(azimuth_error, -self.slew_rate * dt, self.slew_rate * dt)
        elevation_change = np.clip(elevation_error, -self.slew_rate * dt, self.slew_rate * dt)

        self.current_azimuth += azimuth_change
        self.current_elevation += elevation_change

        self.current_azimuth = np.clip(self.current_azimuth, *self.azimuth_range)
        self.current_elevation = np.clip(self.current_elevation, *self.elevation_range)

    def get_current_position(self):
        return self.current_azimuth, self.current_elevation

    def compute_tracking_error(self, desired_azimuth, desired_elevation):
        azimuth_error = desired_azimuth - self.current_azimuth
        elevation_error = desired_elevation - self.current_elevation
        return azimuth_error, elevation_error

    def reset(self):
        self.current_azimuth = 0.0
        self.current_elevation = 0.0

### Controls

In [ ]:
class PIDController:
    def __init__(self, kp, ki, kd, dt):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.dt = dt
        self.prev_error = 0
        self.integral = 0

    def update(self, setpoint, measured_value):
        error = setpoint - measured_value
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt

        output = (self.kp * error) + (self.ki * self.integral) + (self.kd * derivative)

        self.prev_error = error
        return output

class ControlSystem:
    def __init__(self, kp, ki, kd, dt):
        self.pid_controller = PIDController(kp, ki, kd, dt)

    def compute_control_signal(self, desired_angle, current_angle):
        return self.pid_controller.update(desired_angle, current_angle)

    def apply_motor_dynamics(self, control_signal, max_slew_rate):
        # Simulate motor dynamics with a simple slew rate limit
        return np.clip(control_signal, -max_slew_rate, max_slew_rate)

### Rocket

In [ ]:
class Rocket:
    def __init__(self, initial_position=0.0, initial_velocity=0.0, thrust=0.0, mass=1.0):
        self.position = initial_position  # in meters
        self.velocity = initial_velocity    # in meters per second
        self.thrust = thrust                # in newtons
        self.mass = mass                    # in kilograms
        self.acceleration = 0.0             # in meters per second squared

    def update_dynamics(self, time_step):
        # Update acceleration based on thrust and mass
        self.acceleration = self.thrust / self.mass

        # Update velocity and position using simple kinematics
        self.velocity += self.acceleration * time_step
        self.position += self.velocity * time_step

    def get_state(self):
        return {
            'position': self.position,
            'velocity': self.velocity,
            'acceleration': self.acceleration
        }

### Tracking

In [ ]:
import numpy as np
import pandas as pd

class TrackingAlgorithm:
    def __init__(self, dist_to_pad):
        self.dist_to_pad = dist_to_pad  # Distance to the pad in meters

    def compute_desired_angles(self, rocket_position, rocket_altitude):
        """
        Compute the desired azimuth and elevation angles for the antenna based on the rocket's position.

        Parameters:
        - rocket_position: A tuple (x, y) representing the rocket's horizontal position in meters.
        - rocket_altitude: The altitude of the rocket in meters.

        Returns:
        - azimuth: The desired azimuth angle in degrees.
        - elevation: The desired elevation angle in degrees.
        """
        x, y = rocket_position
        dist_to_rocket = np.sqrt(x**2 + y**2 + rocket_altitude**2)

        # Azimuth angle calculation
        azimuth = np.arctan2(y, x)  # radians
        azimuth = np.degrees(azimuth)  # convert to degrees

        # Elevation angle calculation
        elevation = np.arctan2(rocket_altitude, np.sqrt(x**2 + y**2))  # radians
        elevation = np.degrees(elevation)  # convert to degrees

        return azimuth, elevation

    def compute_tracking_error(self, current_angles, desired_angles):
        """
        Compute the tracking error between the current and desired angles.

        Parameters:
        - current_angles: A tuple (current_azimuth, current_elevation) in degrees.
        - desired_angles: A tuple (desired_azimuth, desired_elevation) in degrees.

        Returns:
        - error: A tuple (azimuth_error, elevation_error) in degrees.
        """
        current_azimuth, current_elevation = current_angles
        desired_azimuth, desired_elevation = desired_angles

        azimuth_error = desired_azimuth - current_azimuth
        elevation_error = desired_elevation - current_elevation

        return azimuth_error, elevation_error

    def update_tracking(self, rocket_position, rocket_altitude, current_angles):
        """
        Update the tracking angles based on the rocket's position and altitude.

        Parameters:
        - rocket_position: A tuple (x, y) representing the rocket's horizontal position in meters.
        - rocket_altitude: The altitude of the rocket in meters.
        - current_angles: A tuple (current_azimuth, current_elevation) in degrees.

        Returns:
        - desired_angles: A tuple (desired_azimuth, desired_elevation) in degrees.
        - tracking_error: A tuple (azimuth_error, elevation_error) in degrees.
        """
        desired_angles = self.compute_desired_angles(rocket_position, rocket_altitude)
        tracking_error = self.compute_tracking_error(current_angles, desired_angles)

        return desired_angles, tracking_error

## Utils

### Data_Loader

In [ ]:
import pandas as pd

def load_flight_data(file_path):
    """
    Load flight data from a CSV file.

    Parameters:
    - file_path: str, path to the CSV file.

    Returns:
    - DataFrame containing the flight data.
    """
    try:
        data = pd.read_csv(file_path)
        return data
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        return None

def preprocess_flight_data(df, time_col='Time', altitude_col='Altitude', velocity_col='Velocity'):
    """
    Preprocess flight data to extract relevant columns and rename them.

    Parameters:
    - df: DataFrame, the flight data.
    - time_col: str, name of the time column.
    - altitude_col: str, name of the altitude column.
    - velocity_col: str, name of the velocity column.

    Returns:
    - DataFrame with relevant columns renamed.
    """
    if df is not None:
        df = df[[time_col, altitude_col, velocity_col]].copy()
        df.rename(columns={time_col: 'Time', altitude_col: 'Altitude', velocity_col: 'Velocity'}, inplace=True)
        return df
    else:
        print("DataFrame is None, cannot preprocess.")
        return None

## Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_az_el(time, azimuth, elevation):
    plt.figure(figsize=(12, 6))

    plt.subplot(2, 1, 1)
    plt.plot(time, azimuth, label='Azimuth (°)', color='tab:blue')
    plt.title('Azimuth vs Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Azimuth (°)')
    plt.grid()
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(time, elevation, label='Elevation (°)', color='tab:orange')
    plt.title('Elevation vs Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Elevation (°)')
    plt.grid()
    plt.legend()

    plt.tight_layout()
    plt.show()

def plot_tracking_error(time, tracking_error):
    plt.figure(figsize=(10, 6))
    plt.plot(time, tracking_error, label='Tracking Error (°)', color='tab:red')
    plt.title('Tracking Error vs Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Tracking Error (°)')
    plt.grid()
    plt.legend()
    plt.show()

def plot_rocket_position(time, position):
    plt.figure(figsize=(10, 6))
    plt.plot(time, position, label='Rocket Position (m)', color='tab:green')
    plt.title('Rocket Position vs Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Position (m)')
    plt.grid()
    plt.legend()
    plt.show()

def plot_antenna_vs_rocket(rocket_position, antenna_position):
    plt.figure(figsize=(10, 6))
    plt.plot(rocket_position, label='Rocket Position (m)', color='tab:green')
    plt.plot(antenna_position, label='Antenna Position (m)', color='tab:blue')
    plt.title('Antenna vs Rocket Position')
    plt.xlabel('Time (s)')
    plt.ylabel('Position (m)')
    plt.grid()
    plt.legend()
    plt.show()

def plot_signal_strength(time, signal_strength):
    plt.figure(figsize=(10, 6))
    plt.plot(time, signal_strength, label='Signal Strength (dB)', color='tab:purple')
    plt.title('Signal Strength vs Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Signal Strength (dB)')
    plt.grid()
    plt.legend()
    plt.show()

def plot_results(rocket, antenna):
    """
    Generates all the required plots from the simulation results.

    Args:
        rocket (Rocket): The rocket object containing its simulation history.
        antenna (Antenna): The antenna object containing its simulation history.
    """
    # Assuming the objects have attributes that store the history of values
    # e.g., rocket.time_history, rocket.position_history, antenna.azimuth_history, etc.

    # Check if history data exists
    if not hasattr(rocket, 'time_history') or not rocket.time_history:
        print("Simulation has not been run or no history was logged. Cannot plot results.")
        return

    time = rocket.time_history

    # Plot Azimuth and Elevation
    if hasattr(antenna, 'azimuth_history') and hasattr(antenna, 'elevation_history'):
        plot_az_el(time, antenna.azimuth_history, antenna.elevation_history)

    # Plot Tracking Error
    if hasattr(antenna, 'tracking_error_history'):
        plot_tracking_error(time, antenna.tracking_error_history)

    # Plot Rocket Position (Altitude)
    if hasattr(rocket, 'altitude_history'):
        plot_rocket_position(time, rocket.altitude_history)

    # Note: Additional plots like 'plot_antenna_vs_rocket' and 'plot_signal_strength'
    # can be called here if the necessary data is available in the rocket and antenna objects.

# Config

In [ ]:
# Configuration settings for the rocket tracker simulator

# Tracking algorithm parameters
TRACKING_ALGORITHM = {
    'type': 'KrakenSdr',  # Type of tracking algorithm
    'filters': ['Kalman', 'alpha-beta'],  # Filters to be used in the tracking algorithm
}

# Control system parameters
CONTROL_SYSTEM = {
    'type': 'PID',  # Type of control system
    'Kp': 1.0,  # Proportional gain
    'Ki': 0.1,  # Integral gain
    'Kd': 0.05,  # Derivative gain
    'max_slew_rate': 10.0,  # Maximum slew rate for the motors
    'latency': 0.1,  # Latency in seconds
}

# Antenna parameters
ANTENNA = {
    'azimuth_range': (0, 360),  # Azimuth range in degrees
    'elevation_range': (0, 90),  # Elevation range in degrees
}

# Visualization settings
VISUALIZATION = {
    'show_3D': True,  # Whether to show 3D visualizations
    'show_2D': True,  # Whether to show 2D visualizations
    'plot_tracking_errors': True,  # Whether to plot tracking errors
}

# Rocket dynamics parameters
ROCKET_DYNAMICS = {
    'thrust_phase_duration': 30,  # Duration of the thrust phase in seconds
    'descent_phase_duration': 60,  # Duration of the descent phase in seconds
}

# Communication simulation parameters
COMMUNICATION = {
    'signal_strength': -100,  # Initial signal strength in dBm
    'free_space_path_loss': 100,  # Free space path loss in dB
    'yagi_gain_pattern': 'standard',  # Yagi gain pattern
}

# Main

In [ ]:
def main():
    # Load flight data
    flight_data_60k = load_flight_data('/content/FlightData60k.csv')
    flight_data_10k = load_flight_data('/content/MRT Porthos official Flight data 2023.csv')

    # Initialize rocket and antenna
    rocket = Rocket(flight_data_60k)
    antenna = Antenna()

    # Initialize tracking algorithm and control system
    tracking_algorithm = TrackingAlgorithm(rocket, antenna)
    control_system = ControlSystem(antenna)

    # Simulation loop
    for time in np.arange(0, rocket.total_time, rocket.time_step):
        # Update rocket position
        rocket.update_position(time)

        # Compute desired angles for the antenna
        desired_angles = tracking_algorithm.compute_desired_angles()

        # Apply control algorithm to move the antenna
        control_system.move_antenna(desired_angles)

        # Log results (optional)
        # ...

    # Plot results
    plot_results(rocket, antenna)

if __name__ == "__main__":
    main()

TypeError: TrackingAlgorithm.__init__() takes 2 positional arguments but 3 were given

# Tests

## Test_Antenna

In [ ]:
import unittest

class TestAntenna(unittest.TestCase):

    def setUp(self):
        self.antenna = Antenna()

    def test_initial_angle(self):
        self.assertEqual(self.antenna.angle, 0)

    def test_set_angle(self):
        self.antenna.set_angle(45)
        self.assertEqual(self.antenna.angle, 45)

    def test_angle_limits(self):
        with self.assertRaises(ValueError):
            self.antenna.set_angle(361)  # Exceeds max limit
        with self.assertRaises(ValueError):
            self.antenna.set_angle(-1)   # Exceeds min limit

    def test_tracking_error(self):
        self.antenna.set_angle(30)
        error = self.antenna.calculate_tracking_error(45)
        self.assertEqual(error, 15)

    def test_lag_in_tracking(self):
        self.antenna.set_angle(30)
        self.antenna.update_angle(45, lag=5)
        self.assertNotEqual(self.antenna.angle, 45)  # Should not instantly reach target angle

if __name__ == '__main__':
    unittest.main()

## Test_Rocket

In [ ]:
import unittest
import pandas as pd
from src.components.rocket import Rocket

class TestRocket(unittest.TestCase):

    def setUp(self):
        # Sample data for testing
        self.time = [0, 1, 2, 3, 4, 5]
        self.altitude = [0, 100, 200, 300, 400, 500]
        self.velocity = [0, 20, 40, 60, 80, 100]
        self.rocket = Rocket()

    def test_position_over_time(self):
        # Simulate rocket position
        self.rocket.simulate(self.time, self.altitude, self.velocity)
        expected_positions = [0, 100, 200, 300, 400, 500]
        self.assertEqual(self.rocket.position, expected_positions)

    def test_velocity_over_time(self):
        # Simulate rocket velocity
        self.rocket.simulate(self.time, self.altitude, self.velocity)
        expected_velocities = [0, 20, 40, 60, 80, 100]
        self.assertEqual(self.rocket.velocity, expected_velocities)

    def test_acceleration_calculation(self):
        # Test acceleration calculation
        self.rocket.simulate(self.time, self.altitude, self.velocity)
        expected_accelerations = [20, 20, 20, 20, 20]  # Constant acceleration
        self.assertEqual(self.rocket.acceleration, expected_accelerations)

if __name__ == '__main__':
    unittest.main()

# README

# Rocket Tracker Simulator

## Overview
The Rocket Tracker Simulator is a comprehensive simulation tool designed to model the dynamics of a rocket and its tracking antenna. The simulator integrates flight data from actual rocket flights and simulates the behavior of the antenna as it tracks the rocket's position in real-time. This project aims to provide insights into antenna pointing mechanisms, tracking algorithms, and control systems.

## Features
- Simulates rocket dynamics, including position, velocity, and acceleration.
- Models the antenna's pointing mechanism with angle, lag, and error calculations.
- Implements tracking algorithms to compute desired antenna angles based on rocket data.
- Provides a control system for moving the antenna to the desired position using PID control.
- Visualizes simulation outputs in both 2D and 3D formats.

## Project Structure
```
rocket-tracker-simulator
├── data
│   ├── FlightData60k.csv
│   └── MRT Porthos official Flight data 2023.csv
├── notebooks
│   └── Sim.ipynb
├── src
│   ├── __init__.py
│   ├── main.py
│   ├── config.py
│   ├── components
│   │   ├── __init__.py
│   │   ├── rocket.py
│   │   ├── antenna.py
│   │   ├── tracking.py
│   │   └── control.py
│   ├── utils
│   │   ├── __init__.py
│   │   └── data_loader.py
│   └── visualization
│       ├── __init__.py
│       └── plotting.py
├── tests
│   ├── __init__.py
│   ├── test_rocket.py
│   └── test_antenna.py
├── requirements.txt
└── README.md
```

## Installation
To set up the project, clone the repository and install the required dependencies:

```bash
git clone <repository-url>
cd rocket-tracker-simulator
pip install -r requirements.txt
```

## Usage
1. Load flight data from the CSV files located in the `data` directory.
2. Run the Jupyter notebook `notebooks/Sim.ipynb` to explore initial visualizations and data processing.
3. Use the `src/main.py` file to execute the simulation, which integrates all components of the project.

## Contributing
Contributions are welcome! Please feel free to submit a pull request or open an issue for any suggestions or improvements.

## License
This project is licensed under the MIT License. See the LICENSE file for details.